## Notes

For notes refer to: [readme.md](readme.md) (with a markdown viewer of course)

## Libraries

In [1]:
from web3 import Web3
from web3.middleware import geth_poa_middleware

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [20]:
from datetime import datetime

In [3]:
import os
import json
import requests

In [4]:
import plotly.express as px

## Inits

### Web3

In [5]:
w3 = Web3(Web3.HTTPProvider("https://api.avax.network/ext/bc/C/rpc"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.isConnected()

True

### Addresses

In [6]:
cra = w3.toChecksumAddress("0xa32608e873f9ddef944b24798db69d80bbb4d1ed")
tus = w3.toChecksumAddress("0xf693248f96fe03422fea95ac0afbbbc4a8fdd172")
crabada=w3.toChecksumAddress("0x1b7966315ef0259de890f38f1bdb95acc03cacdd")
marketplace=w3.toChecksumAddress("0x7E8DEef5bb861cF158d8BdaAa1c31f7B49922F49")
game=w3.toChecksumAddress("0x82a85407BD612f52577909F4A58bfC6873f14DA8")

### Contract ABIs

In [7]:
abi = json.loads(
'''
[{
"inputs": [], "name": "totalSupply", 
"outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}], 
"stateMutability": "view", 
"type": "function"
},
{
"inputs": [],
"name": "price",
"outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
"stateMutability": "view",
"type": "function"
},
{
"inputs": [{"internalType": "uint256", "name": "tokenId", "type": "uint256"}],
"name": "tokenURI", 
"outputs": [{"internalType": "string", "name": "", "type": "string"}],
"stateMutability": "view",
"type": "function"
},
{
"constant": true,
"inputs": [{"internalType": "uint256", "name": "tokenId", "type": "uint256"}],
"name": "ownerOf",
"outputs": [{"internalType": "address", "name": "", "type": "address"}],
"payable": false,
"stateMutability": "view",
"type": "function"
}
]
''')

In [8]:
crabContract = w3.eth.contract(address=crabada, abi=abi)

### Data

In [8]:
df = pd.read_pickle("./data/raw/crabAPIdata.pkl")

In [9]:
# get data into columns
crabs = df["data"].apply(lambda x: pd.Series(x["result"]))
crabs.set_index('id', inplace=True)
crabs.set_index('crabada_id', inplace=True)
crabs.head()

,crabada_type,type,is_origin,is_genesis,legend_number,pure_number,stage,dna,name,description,...,order_time,owner,owner_full_name,owner_photo,owner_nft_avatar,breed_count,created_at,updated_at,breeding_fee,is_favorite
crabada_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,GENESIS,1,1.0,0.0,6.0,1,9063925442313111610762316178811172860734019784...,Crabada 1,None,...,None,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,None,None,None,0,2021-11-04T04:04:09.973688,2021-11-13T07:59:06.249173,"{'CRA': 0, 'TUS': 0}",0
2,2.0,GENESIS,1,1.0,0.0,6.0,1,9081593912960895454058149153818602920431322016...,Crabada 2,None,...,None,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,None,None,None,0,2021-11-04T04:04:09.973688,2021-11-13T07:59:06.249173,"{'CRA': 0, 'TUS': 0}",0
3,2.0,GENESIS,1,1.0,0.0,6.0,1,9099262383608679297353982128826032980128624248...,Crabada 3,None,...,None,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,None,None,None,0,2021-11-04T04:04:09.973688,2021-11-13T07:59:06.249173,"{'CRA': 0, 'TUS': 0}",0
4,2.0,GENESIS,1,1.0,0.0,6.0,1,9116930854256463140649815103833463039825926481...,Crabada 4,None,...,None,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,None,None,None,0,2021-11-04T04:04:09.973688,2021-11-13T07:59:06.249173,"{'CRA': 0, 'TUS': 0}",0
5,2.0,GENESIS,1,1.0,0.0,6.0,1,9134599324904246983945648078840893099523228713...,Crabada 5,None,...,None,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,None,None,None,0,2021-11-04T04:04:09.973688,2021-11-13T07:59:06.249173,"{'CRA': 0, 'TUS': 0}",0


In [11]:
crabs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10773 entries, 1 to 10773
Data columns (total 58 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   crabada_type      10059 non-null  float64
 1   type              10059 non-null  object 
 2   is_origin         10773 non-null  int64  
 3   is_genesis        10059 non-null  float64
 4   legend_number     10059 non-null  float64
 5   pure_number       10059 non-null  float64
 6   stage             10773 non-null  int64  
 7   dna               10059 non-null  object 
 8   name              10059 non-null  object 
 9   description       0 non-null      object 
 10  photo             10059 non-null  object 
 11  atlas_photo       10059 non-null  object 
 12  crabada_class     10059 non-null  float64
 13  class_id          10059 non-null  float64
 14  class_name        10059 non-null  object 
 15  crabada_subclass  10059 non-null  float64
 16  birthday          10773 non-null  int64 

In [12]:
# save
crabs.to_pickle("./data/raw/crabData.pkl")

In [13]:
crabs.type.unique()

array(['GENESIS', 'NORMAL', None], dtype=object)

In [14]:
crabs.legend_number.unique()

array([ 0., nan,  3.,  1.,  2.])

In [15]:
crabs.pure_number.unique()

array([ 6.,  5.,  4.,  3.,  2., nan,  1.,  0.])

In [16]:
crabs.stage.unique()

array([1, 0])

In [17]:
crabs.crabada_type.unique()

array([ 2.,  1., nan])

In [90]:
crabs.type.unique()

array(['GENESIS', 'NORMAL', None], dtype=object)

In [18]:
crabs.is_genesis.unique()

array([ 1.,  0., nan])

In [26]:
datetime.utcfromtimestamp(1639924393).strftime('%Y-%m-%d %H:%M:%S')

'2021-12-19 14:33:13'

In [30]:
crabs["birthtime"] = crabs["birthday"].apply(lambda x: pd.to_datetime(x, unit="s"))

## Explore Data

### Owners

<u>Based on SnowTrace</u>:  

0x82a85407BD612f52577909F4A58bfC6873f14DA8: `Crabada Game`  
0x1fA283b8C14e2d33e699cCe56Bf32b7cb2DB67d8:  Perhaps the special event.  
0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d:  
0x7E8DEef5bb861cF158d8BdaAa1c31f7B49922F49:  `Market Place`

In [73]:
owner_count = crabs.groupby("owner").size().to_frame(name="count").sort_values(by="count", ascending=False).reset_index()
owner_count #552 unique holders

,owner,count
0,0x82a85407bd612f52577909f4a58bfc6873f14da8,8441
1,0x1fa283b8c14e2d33e699cce56bf32b7cb2db67d8,372
2,0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d,364
3,0xcea85ace606ef0a2166c9de211c5a2af4d14590b,50
4,0x2bf7cd06a44e01cc94a0f7b9dc8b57c6a2d15157,28
...,...,...
547,0x4ae29c5dc1cf53e966bc06eeb829b930e54d228a,1
548,0xb265ac5bc12a97bc359b0ca0870f8b1cec6369e9,1
549,0x294a2b72e20c36a114cf9bbd1c9a22478b83b93b,1
550,0x4c066df09785169d1083ac8c2815e5ea38e919a9,1


In [32]:
# how many have more than 1 crab? #about 292
mt2crabs = owner_count[(owner_count["count"]>=2) & (owner_count["count"]<300)]
mt2crabs

,owner,count
3,0xcea85ace606ef0a2166c9de211c5a2af4d14590b,50
4,0x2bf7cd06a44e01cc94a0f7b9dc8b57c6a2d15157,28
5,0xcccd64fd8976a03ae897fad97249e01fb9170eb6,27
6,0xda0df05be32a391c23cf4a714b606a9109527ea4,25
7,0x4716a689a7b1378884cd1a1f200c110cbed0bc55,24
...,...,...
290,0x6c7bdd8be7d666fbd3a0474f1079c78e1346347f,2
291,0x7c362f61d453ba5d52df45244cdaedec300a4533,2
292,0x4eb42b30032ee25802e51f5d5caba14b741cadf3,2
293,0x4f266f295bad85436ecea51e0ae2afe82b11553b,2


In [33]:
owner_count[owner_count["count"]<300].describe() # median is 2

,count
count,549.000000
mean,2.907104
std,3.966618
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,50.000000


In [34]:
for i in range(20, 3, -1):
    print("{} is in {:.2f} percentile".format(i, stats.percentileofscore(owner_count["count"], i)))

#top 10 percentile has 10 crabs

20 is in 98.46 percentile
19 is in 98.19 percentile
18 is in 98.01 percentile
17 is in 97.83 percentile
16 is in 97.64 percentile
15 is in 97.28 percentile
14 is in 97.10 percentile
13 is in 96.92 percentile
12 is in 96.29 percentile
11 is in 95.74 percentile
10 is in 95.38 percentile
9 is in 94.75 percentile
8 is in 93.84 percentile
7 is in 92.75 percentile
6 is in 90.67 percentile
5 is in 88.04 percentile
4 is in 84.78 percentile


In [35]:
# how many addresses have 10 or more crabs - 27/449 hodlers
owner_count[owner_count["count"]>9]["owner"].count()

27

In [36]:
# how many have more than 7 crab? #about 245
mt7crabs = owner_count[(owner_count["count"]>=7) & (owner_count["count"]<300)]

In [59]:
fig = px.bar(
    mt7crabs,
    x="owner",
    y="count",
    color="count",
    title="Crab Distribution vs Owner"
)
# overwrite tick labels    
fig.update_layout(
    xaxis = {
     'tickmode': 'array',
     'tickvals': list(range(len(mt7crabs))),
     'ticktext': mt2crabs["owner"].str.slice(stop=7).to_list(),
    }
)
fig.show()

In [63]:
# Owners with Genesis Crabs (0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d is a game contract)
# So apparently the bulk of genesis is via the bootstrap event and are yet disseminated.
crabs[crabs.type=="GENESIS"].groupby("owner").size()

owner
0x06dacefefb7d9ecb7091a8adaa6238703819588d     1
0x3b7631f8e3428deab77634bf799b622ce412e9ea     1
0x82a85407bd612f52577909f4a58bfc6873f14da8     7
0xb95afb5318e340f3afdf2dde2d9c3fc041c6ae2d    54
0xcef785a10930dfbb5c17eb1746ba11359b128a78     1
dtype: int64

### Crab Data

In [89]:
px.histogram(crabs[crabs.pure_number.notnull()], x="birthtime", color="pure_number",title="Distribution over time of Crabs born")

In [91]:
px.histogram(crabs[crabs.pure_number.notnull()], x="birthtime", color="class_name",title="Distribution over time of Crabs born")

In [118]:
ownerblacklist=["0x82a85407bd612f52577909f4a58bfc6873f14da8".lower(), 
                "0x1fA283b8C14e2d33e699cCe56Bf32b7cb2DB67d8".lower(), 
                    "0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d".lower(),
                    "0x7E8DEef5bb861cF158d8BdaAa1c31f7B49922F49".lower()]

In [140]:
crabs["shortowner"] = crabs["owner"].str.slice(stop=7)

In [144]:
px.bar(crabs[(~crabs.owner.isin(ownerblacklist)) & (crabs.pure_number.notnull())], 
                 x="shortowner", 
                   color="class_name", 
                   title="Crab Count By Owner"
              )

# overwrite tick labels    
# fig.update_layout(
#     xaxis = {
#      #'tickmode': 'array',
#      'tickvals': list(range(len(crabs[~crabs.owner.isin(ownerblacklist)]))),
#      'ticktext': crabs[~crabs.owner.isin(ownerblacklist)]["owner"].str.slice(stop=7).to_list(),
#     }
# )

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

Eggs take 5 days to hatch. After 4th Nov, no crabs born till 11th Nov.

In [41]:
print("{} initial crabs before 4th Nov.".format(370+1680+3950))

6000 initial crabs before 4th Nov


In [56]:
# how many genesis crabs (64, I did not read the medium posts or track the stuff :P)
print(crabs["type"].value_counts())
print()
print(crabs[crabs.birthtime < "2021-11-5"]["type"].value_counts())

NORMAL     9995
GENESIS      64
Name: type, dtype: int64

NORMAL     5936
GENESIS      64
Name: type, dtype: int64


### Wallet Revenue

In [59]:
# looting data
requests.get("https://idle-api.crabada.com/public/idle/mines?looter_address={wallet}&status=close&limit=10000".format(
                    wallet="0x3660f03882ff584B1d97d8A7adeD8163db5de610"
                )
            ).json()

{'error_code': None,
 'message': None,
 'result': {'totalRecord': 222,
  'totalPages': 1,
  'page': 1,
  'limit': 10000,
  'data': [{'game_id': 43406,
    'start_time': 1637961072,
    'end_time': 1637975472,
    'cra_reward': 4125000000000000000,
    'tus_reward': 334125000000000000000,
    'miner_cra_reward': 1687500000000000000,
    'miner_tus_reward': 136687500000000000000,
    'looter_cra_reward': 2737500000000000000,
    'looter_tus_reward': 221737500000000000000,
    'estimate_looter_win_cra': 2737500000000000000,
    'estimate_looter_win_tus': 221737500000000000000,
    'estimate_looter_lose_cra': 300000000000000000,
    'estimate_looter_lose_tus': 24300000000000000000,
    'estimate_miner_lose_cra': 1687500000000000000,
    'estimate_miner_lose_tus': 136687500000000000000,
    'estimate_miner_win_cra': 4125000000000000000,
    'estimate_miner_win_tus': 334125000000000000000,
    'round': 0,
    'team_id': 2090,
    'owner': '0x6e7609bbc3701f42ea56e068841cdc0955e7feec',
    'de

In [58]:
# mining data
requests.get("https://idle-api.crabada.com/public/idle/mines?user_address={wallet}&status=close&limit=10000".format(
                    wallet="0x3660f03882ff584B1d97d8A7adeD8163db5de610"
                )
            ).json()

{'error_code': None,
 'message': None,
 'result': {'totalRecord': 233,
  'totalPages': 1,
  'page': 1,
  'limit': 10000,
  'data': [{'game_id': 143901,
    'start_time': 1639517878,
    'end_time': 1639532278,
    'cra_reward': 3750000000000000000,
    'tus_reward': 303750000000000000000,
    'miner_cra_reward': 3750000000000000000,
    'miner_tus_reward': 303750000000000000000,
    'looter_cra_reward': 300000000000000000,
    'looter_tus_reward': 24300000000000000000,
    'estimate_looter_win_cra': 2737500000000000000,
    'estimate_looter_win_tus': 221737500000000000000,
    'estimate_looter_lose_cra': 300000000000000000,
    'estimate_looter_lose_tus': 24300000000000000000,
    'estimate_miner_lose_cra': 1312500000000000000,
    'estimate_miner_lose_tus': 106312500000000000000,
    'estimate_miner_win_cra': 3750000000000000000,
    'estimate_miner_win_tus': 303750000000000000000,
    'round': 0,
    'team_id': 1199,
    'owner': '0x3660f03882ff584b1d97d8a7aded8163db5de610',
    'def

In [60]:
# out of game data
requests.get("https://idle-api.crabada.com/public/idle/crabadas/out-game?user_address={wallet}&order=desc&limit=10000".format(
                    wallet="0x3660f03882ff584B1d97d8A7adeD8163db5de610"
                )
            ).json()

{'error_code': None,
 'message': None,
 'result': {'totalRecord': 0,
  'totalPages': 1,
  'page': 1,
  'limit': 10000,
  'data': [],
  'total_crabada': '0'}}

In [63]:
# for borrowed data
requests.get("https://idle-api.crabada.com/public/idle/crabadas/for-lending?user_address={wallet}&limit=10000".format(
                    wallet="0x3660f03882ff584B1d97d8A7adeD8163db5de610"
                )
            ).json()

{'error_code': None,
 'message': None,
 'result': {'totalRecord': 1,
  'totalPages': 1,
  'page': 1,
  'limit': 10000,
  'data': [{'crabada_id': 9047,
    'id': 9047,
    'crabada_name': 'Crabada 9047',
    'owner': '0x3660f03882ff584b1d97d8a7aded8163db5de610',
    'crabada_type': 1,
    'crabada_class': 2,
    'class_id': 2,
    'class_name': 'SUNKEN',
    'is_origin': 0,
    'is_genesis': 0,
    'crabada_status': 'WAS_BORROWED',
    'borrower_address': '0x118613c8c5c6445bb7419e1ca329a78ef95e2123',
    'borrow_time': 1639561733,
    'game_id': 147262,
    'legend_number': 0,
    'pure_number': 2,
    'photo': '9047.png',
    'hp': 121,
    'speed': 30,
    'damage': 54,
    'critical': 38,
    'armor': 29,
    'battle_point': 204,
    'time_point': 68,
    'mine_point': 68}]}}

In [40]:
# rented teams
r = requests.get("https://idle-api.crabada.com/public/idle/teams?user_address={wallet}".format(
    wallet="0x3660f03882ff584B1d97d8A7adeD8163db5de610")).json()

In [57]:
# rented crab history
requests.get("https://idle-api.crabada.com/public/idle/crabadas/lending/history?crabada_id={id}&limit=10000".format(
    id="3033")).json()

{'error_code': None,
 'message': None,
 'result': {'totalRecord': 0,
  'totalPages': 1,
  'page': 1,
  'limit': 10000,
  'data': []}}

In [70]:
# search crabs
r = requests.get("https://api.crabada.com/public/crabada/selling?limit=2000&stage=1&orderBy=price&order=asc").json()
print("num of records {}".format(len(r["result"]["data"])))

num of records 302


**crabada resources: https://github.com/crabada-resources**  
- Looting Stats: https://idle-api.crabada.com/public/idle/
- Crab Finder: https://api.crabada.com/public/crabada/selling
- Breeding Calculator: https://api.crabada.com/public/crabada/family/${eggid} (calculate egg)
- Should I Buy:
- Search Stats: https://api.crabada.com/public/crabada/selling

### Crabs

In [19]:
crabNowners = crabs_df.join(owners_df)
crabNowners.head()

,birthday,class,pure,legend,origin,genesis,hp,speed,damage,armor,critical,owner
1,1.635778e+09,SURGE,6.0,0.0,1.0,1.0,150.0,26.0,50.0,37.0,39.0,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
2,1.635778e+09,SURGE,6.0,0.0,1.0,1.0,150.0,26.0,50.0,37.0,39.0,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
3,1.635778e+09,SURGE,6.0,0.0,1.0,1.0,150.0,26.0,50.0,37.0,39.0,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
4,1.635778e+09,SURGE,6.0,0.0,1.0,1.0,150.0,26.0,50.0,37.0,39.0,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d
5,1.635778e+09,SURGE,6.0,0.0,1.0,1.0,150.0,26.0,50.0,37.0,39.0,0xb95afB5318E340F3aFDF2ddE2D9c3Fc041C6aE2d


In [20]:
crabNowners.to_pickle("./data/raw/crabNowners.pkl")